In [ ]:
from huggingface_hub import login
login()

In [1]:

%pip install accelerate peft bitsandbytes transformers trl jsonlines 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 13.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.0/124.0 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 2.0 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 39.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 3.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 1.

In [2]:
pip install git+https://github.com/huggingface/transformers


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nxx4e4fi
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-nxx4e4fi
  Resolved https://github.com/huggingface/transformers to commit 5bbf6712767a84ac4c4de1a5525a1b4e697064f7
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 7.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 26.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 14.7 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-4.35.0.dev0-py3-none-any.whl size=7884086 sha256=d15fa6f3e97d90dd145f3778654a0aa453b8b3c9e82fc15bbf04138222b4b3f6
  Stored in directory: /tmp/pip-ephem-wh

In [3]:
!source llm/bin/activate

In [1]:
import os
import torch
import jsonlines
from pprint import pprint
import datasets
import pandas as pd
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig
from trl import SFTTrainer

In [2]:
# Model from Hugging Face hub
model_name = "microsoft/DialoGPT-large"

# New instruction dataset
bargains_dataset = "craigslist_bargains"

# Fine-tuned model
new_model = "llama-2-7b-chat-bargains"

# New Section

In [3]:
dataset = load_dataset(bargains_dataset, split="train")

Using custom data configuration default


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/5247 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/838 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/597 [00:00<?, ? examples/s]

Dataset craigslist_bargains downloaded and prepared to /root/.cache/huggingface/datasets/craigslist_bargains/default/1.1.0/9e4e64d13ea36e435a8f56f9305d3c771115909b7fe83269bf7cd8dd40302338. Subsequent calls will reuse this data.


In [ ]:
dataset

In [ ]:
dataset['utterance'][0]   

In [ ]:
dataset['agent_turn'][0]

In [ ]:
example =dataset[0]
example

In [1]:
def create_prompt_dict(data):
    # Extract data
    agent_info = data['agent_info']
    agent_turn = data['agent_turn']
    dialogue_acts = data['dialogue_acts']
    utterances = data['utterance']
    items = data['items']

    # Initialize variables to store the conversation
    conversation_prompt = []
    

    # Iterate through the data and assign roles based on agent_turn
    for i in agent_turn:
        if agent_turn[i] == 0:
            conversation_prompt.append({
                agent_info['Role'][0]: utterances[i]
            })
        elif agent_turn[i] == 1:
            conversation_prompt.append({
                agent_info['Role'][1]: utterances[i]
            })

       

    # Additional information
    agent_role = ['buyer' if turn == 0 else 'seller' for turn in agent_turn]
    agent_target = agent_info['Target']
    dialogue_intent = dialogue_acts['intent']
    dialogue_price = dialogue_acts['price']
    item_category = items['Category'][0]
    item_images = items['Images']
    item_price = items['Price'][0]
    item_description = items['Description']
    item_title = items['Title']

    # Construct the full prompt in dictionary format
    full_prompt_dict = {
        "Conversation": conversation_prompt,

        "Items": {
            "Title": item_title,
            "Category": item_category,
            "Price": item_price,
            "Target": agent_target,
            "Description": item_description,

            }
    }

    return full_prompt_dict



In [ ]:
formatted_prompts = list(map(create_prompt_dict, dataset))

In [ ]:
with jsonlines.open(f'bargains_processed.jsonl', 'w') as writer:
    writer.write_all(formatted_prompts)

In [ ]:
pprint(formatted_prompts[0])

{'Conversation': [{'buyer': 'Hi, not sure if the charger would work for my '
                            'car. Can you sell it to me for $5?'},
                  {'seller': 'It will work, i have never seen a car without a '
                             'cigarette lighter port.\\'},
                  {'buyer': 'Hi, not sure if the charger would work for my '
                            'car. Can you sell it to me for $5?'},
                  {'seller': 'It will work, i have never seen a car without a '
                             'cigarette lighter port.\\'},
                  {'buyer': 'Hi, not sure if the charger would work for my '
                            'car. Can you sell it to me for $5?'},
                  {'seller': 'It will work, i have never seen a car without a '
                             'cigarette lighter port.\\'},
                  {'buyer': 'Hi, not sure if the charger would work for my '
                            'car. Can you sell it to me for $5?'},
       

In [ ]:


# Define the function to convert data to a string prompt
def convert_data_to_prompt(data):
    # Extract conversation and item details
    conversation = data['Conversation']
    items = data['Items']

    # Initialize the prompt as an empty string
    prompt = """<<SYS>> You're are a expert seller,your are having a conversation with a buyer  and negotiating  about the price, you are not allowed to change your roleadd sell the item less then target range. this is the detail discription and information related to the item """



    # Append item details to the prompt
    category = items['Category'][0]
    price = items['Price']
    target = items['Target']
    description = items['Description'][0]
    title = items['Title'][0]
    

    prompt += f"Category: {category}, "
    prompt += f"Price: {price},"
    prompt += f"Target: {target}, "
    prompt += f"Title: {title}, "
    prompt += f"Description: {description} <</SYS>> "
    
    
    # Append conversation to the prompt
    for dialogue in conversation:
        role = list(dialogue.keys())[0]
        utterance = dialogue[role]
        if role == 'buyer':
            prompt += f"[INST] Buyer: {utterance} [/INST] "
        elif role == 'seller':
            prompt += f" Seller: {utterance} "

    return prompt




In [ ]:
prompts = list(map(convert_data_to_prompt, formatted_prompts))

In [ ]:
pprint(prompts[0])

list["<<SYS>> You're are a expert seller,your are having a conversation with a buyer  and negotiating  about the price, you are not allowed to change your roleadd sell the item less then target range. this is the detail discription and information related to the item Category: p, Price: 10.0,Target: [7.0, 10.0], Title: Verizon Car Charger with Dual Output Micro USB and LED Light, Description: Charge two devices simultaneously on the go. This vehicle charger with an additional USB port delivers enough power to charge two devices at once. The push-button activated LED connector light means no more fumbling in the dark trying to connect your device. Auto Detect IC Technology automatically detects the device type and its specific charging needs for improved compatibility. And the built-in indicator light illuminates red to let you know the charger is receiving power and the power socket is working properly. <</SYS>> ", '']


In [ ]:
prompts[0]

In [ ]:
train_dataset = datasets.Dataset.from_pandas(pd.DataFrame(data=prompts, columns=['Prompt']))

In [ ]:
train_dataset.save_to_disk('train_dataset')

In [4]:
train_dataset = datasets.load_from_disk('train_dataset')

In [5]:
train_dataset

Dataset({
    features: ['Prompt'],
    num_rows: 5247
})

In [6]:

################################################################################
# QLoRA parameters
################################################################################

# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

################################################################################
# bitsandbytes parameters
################################################################################

# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

################################################################################
# TrainingArguments parameters
################################################################################

# Output directory where the model predictions and checkpoints will be stored
output_dir = "./results"

# Number of training epochs
num_train_epochs = 1

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 4

# Batch size per GPU for evaluation
per_device_eval_batch_size = 4

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 1

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 0

# Log every X updates steps
logging_steps = 25

################################################################################
# SFT parameters
################################################################################

# Maximum sequence length to use
max_seq_length = None

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = 'auto'

In [7]:


# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="Prompt",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.9/dist-packages/trl/trainer/sft_trainer.py:173: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


  0%|          | 0/6 [00:00<?, ?ba/s]

In [8]:
import torch
import os

# Set PYTORCH_CUDA_ALLOC_CONF environment variable
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "caching_allocator"

In [9]:
torch.cuda.empty_cache()

In [10]:
trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


OutOfMemoryError: CUDA out of memory. Tried to allocate 74.00 MiB. GPU 0 has a total capacty of 15.89 GiB of which 46.69 MiB is free. Process 240003 has 15.84 GiB memory in use. Of the allocated memory 15.17 GiB is allocated by PyTorch, and 166.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
    
)


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map='auto',
    offload_folder="offload"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
    base_model,
    padding_side="left",
    add_eos_token=True,
    add_bos_token=True,
)
tokenizer.pad_token = tokenizer.eos_token


In [ ]:
peft_params = LoraConfig(
    lora_alpha=8,
    lora_dropout=0.1,
    r=4,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=100,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    dataset_text_field="Prompt",
    peft_config=peft_params,
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("bargains")

In [ ]:
path = "/content/bargains"

In [ ]:
finetuned_slightly_model = AutoModelForCausalLM.from_pretrained(path, local_files_only=True)


In [ ]:
#clear cache
#torch.cuda.empty_cache()

In [ ]:
import torch 


In [ ]:
torch.cuda.is_available()

In [ ]:
torch.cuda.current_device()

In [ ]:
torch.cuda.device_count()

In [ ]:
torch.cuda.device(0)

In [ ]:
torch.cuda.get_device_name(0)

In [ ]:
!watch -n 2 nvidia-smi


In [ ]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_reserved(0)/1024**3,1), 'GB')

In [21]:
torch.cuda.empty_cache()